In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [4]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [11]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
          "matbench_expt_gap",
        #"matbench_steels",
    ],
)

2023-01-30 18:53:20 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [12]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [13]:
from collections import defaultdict

In [14]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-30 18:53:22 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 339Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_185322/train.jsonl: file-Pe3CQIgH3awFTMyZ1XJH4VIa


2023-01-30 18:53:25.517 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675101205,
  "events": [
    {
      "created_at": 1675101205,
      "level": "info",
      "message": "Created fine-tune: ft-sPVYJtdz9y5EydDNSJbzRiLY",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-sPVYJtdz9y5EydDNSJbzRiLY",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675101204,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_185322/train.jsonl",
      "id": "file-Pe3CQIgH3awFTMyZ1XJH4VIa",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-01-30 19:49:06 INFO     Recorded fold matbench_expt_gap-0 successfully.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 277Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_194906/train.jsonl: file-cj9LCq3XbA25DyHXW9QKdwUw


2023-01-30 19:49:08.690 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675104548,
  "events": [
    {
      "created_at": 1675104548,
      "level": "info",
      "message": "Created fine-tune: ft-ri54ZnvoALQKm5xYgAsbg1hu",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-ri54ZnvoALQKm5xYgAsbg1hu",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420507,
      "created_at": 1675104548,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_194906/train.jsonl",
      "id": "file-cj9LCq3XbA25DyHXW9QKdwUw",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

Fine tuning failed. Result: {
  "created_at": 1675104548,
  "events": [
    {
      "created_at": 1675104548,
      "level": "info",
      "message": "Created fine-tune: ft-ri54ZnvoALQKm5xYgAsbg1hu",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-ri54ZnvoALQKm5xYgAsbg1hu",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420507,
      "created_at": 1675104548,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_194906/train.jsonl",
      "id": "file-cj9LCq3XbA25DyHXW9QKdwUw",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 16751045

In [9]:
task.scores

{'mae': {'mean': 142.05139272913465,
  'max': 164.16984126984127,
  'min': 116.28548387096774,
  'std': 15.649240031586015},
 'rmse': {'mean': 203.6908550641379,
  'max': 228.16760652350862,
  'min': 178.0547437565019,
  'std': 17.405743765520075},
 'mape': {'mean': 0.10162651241009144,
  'max': 0.11943686321489388,
  'min': 0.08450077530845235,
  'std': 0.011543892832947062},
 'max_error': {'mean': 678.9399999999999,
  'max': 765.0999999999999,
  'min': 565.5999999999999,
  'std': 64.39293750094028},
 '_ipython_canary_method_should_not_exist_': {}}

In [10]:
mb.to_file("gpt_steel_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
